## This notebook will analyse and visualise the results of the cocktail party nightmare as the number of bats in a group increase. The results shown here are what the centre-most bat in the group will be experiencing. 

In [1]:
import re
import glob
import multiprocessing
import pickle
import sys
sys.path.append('..//')

import matplotlib.pyplot as plt 
plt.rcParams['agg.path.chunksize'] = 1000
from extract_results_from_simoutput import *

results_folder = 'D:\\the_cocktail_party_nightmare\\simulations\\effect_of_group_size_noshadowing\\'

In [2]:
group_size = [5,10,20,50,100,200]

In [3]:
result_files = {Nbats :  glob.glob(results_folder+str(Nbats)+'bats_CPN*.pkl') for Nbats in group_size}

In [ ]:
# extract properties of heard echoes 
echoesheard, r_groups , theta_groups, nn_groups, level_groups = process_simulation_outputs(results_folder, group_size,
                                                                                         **{'detected_status':1})
print('Done with heard echoes')
# extract properties of unheard echoes 
echoesnotheard, r_groups_not , theta_groups_not, nn_groups_not, level_groups_not = process_simulation_outputs(results_folder,
                                                                                                           group_size,
                                                                                                           **{'detected_status':0})
print('Done with unheard echoes')


(5, ' starting')
(5, 'group size Done')
(10, ' starting')
(10, 'group size Done')
(20, ' starting')


In [ ]:
#with open(results_folder+'processed_effect_of_groupsize_5to200bats_noshadowing.pkl','rb') as heardechoes:
#    [echoesheard, r_groups , theta_groups, nn_groups, level_groups] = pickle.load(heardechoes)
#with open(results_folder+'processed_effect_of_groupsize_5to200bats_noshadowing_notheard.pkl','rb') as notheardechoes:
#    [echoesnotheard, r_groups_not , theta_groups_not, nn_groups_not, level_groups_not] = pickle.load(notheardechoes)

In [ ]:
# save the output into a pickle after processing:
with open(results_folder+'processed_effect_of_groupsize_5to200bats_noshadowing_notheard.pkl','wb') as processedoutput:
   pickle.dump([echoesheard, r_groups_not , theta_groups_not, nn_groups_not, level_groups_not ], processedoutput)
# save the output into a pickle after processing:
with open(results_folder+'processed_effect_of_groupsize_5to200bats_noshadowing.pkl','wb') as processedoutput:
    pickle.dump([echoesheard, r_groups , theta_groups, nn_groups, level_groups ], processedoutput)

In [ ]:
%matplotlib notebook

In [ ]:
plt.figure(figsize=(8,15))
start_plotnum = 321
plt.title('Neighbour detection map, R, $\\theta$');
for i, nbats in enumerate(group_size):
    plotnum = i + start_plotnum
    plt.subplot(plotnum, projection='polar').set_theta_zero_location('N')
    plt.title('Group size: '+ str(nbats))
    if nbats >20:
        notheard_alpha = 0.01
    else:
        notheard_alpha = 0.4

    plt.plot(np.deg2rad(theta_groups_not[nbats]),r_groups_not[nbats],  'r*', markersize=10,
             alpha=notheard_alpha, label='groupsize = '+str(nbats))
    plt.plot(np.deg2rad(theta_groups[nbats]),r_groups[nbats],  'g*', markersize=10,
             alpha=0.4, label='groupsize = '+str(nbats))
    plt.ylim(0,5)
    plt.xticks(np.radians([0,45,90,135,180]))


In [ ]:
echo_levels = [ level_groups[nbats] for nbats in group_size]
echo_levels_not = [ level_groups_not[nbats] for nbats in group_size]

median_echo_levels = map(lambda X : np.median(X), echo_levels)
median_echo_levels_not = map(lambda X : np.median(X), echo_levels_not)
plt.figure(figsize=(7,7))
plt.violinplot(echo_levels, np.array(group_size)-0.5,'b');
parts = plt.violinplot(echo_levels_not, np.array(group_size)+0.5,'r' )
for ps in parts['bodies']:
    ps.set_facecolor('red')
    ps.set_edgecolor('red')
plt.plot(np.array(group_size)-0.5, median_echo_levels,'b-*',label='heard echoes, median levels')
plt.plot(np.array(group_size)+0.5, median_echo_levels_not,'r-*',label='unheard echoes, median levels')
plt.ylabel('Echo received level at central bat \n dB SPL re 20 $\\mu$Pa', fontsize=15)
plt.xlabel('Group size', fontsize=15) ;plt.xscale('log');plt.grid();plt.ylim(20,80);
plt.yticks(np.arange(20,82,6),fontsize=12)
plt.xticks(group_size, group_size,fontsize=12);
plt.legend()

In [ ]:
# visualise how the nearest neighbours detected changes with group size:
plt.figure(figsize=(8,8))
for nbats in group_size:
    nn, nn_counts = np.unique(np.int64(nn_groups[nbats].flatten()), return_counts=True)
    plt.title('Probability of detection nearest neighbours ')
    plt.plot(nn, nn_counts/float(np.sum(nn_counts)), '*-', label='nbats= '+str(nbats))
plt.ylabel('$P(Neighbour_{n\:detection}$')
plt.xticks([1]+np.arange(5,45,5).tolist());plt.grid()
plt.legend()

In [ ]:
plt.figure()  
for nbats in group_size:
    nn, nn_counts = np.unique(np.int64(nn_groups[nbats].flatten()), return_counts=True)
    plt.title('Cumulative probability of detection $P(\leq Neighbour_{n})$')
    plt.plot(nn, 1-np.cumsum(nn_counts/float(np.sum(nn_counts))), '*-', label='nbats= '+str(nbats))
    plt.xlabel('Neighbour rank')
plt.xticks(range(0,50,5));plt.grid()
plt.legend()

In [ ]:
plt.figure()
angles = [theta_groups[nbats] for nbats in group_size]
angles_not = [theta_groups_not[nbats] for nbats in group_size]
median_angles = map(np.median, angles)
median_notangles = map(np.median, angles_not)
plt.violinplot(angles, vert=False)
plt.violinplot(angles_not, vert=False)
plt.yticks(range(1,7), group_size);plt.xlabel('Echo reception angles, $^{\circ}$')

In [ ]:
plt.figure(figsize=(8,8))
radii_detection = [r_groups[nbats] for nbats in group_size]
radii_nondetection = [r_groups_not[nbats] for nbats in group_size]
median_radius = map(np.median, radii_detection)
median_notradius = map(np.median, radii_nondetection)

plt.violinplot(radii_detection, group_size)
plt.violinplot(radii_nondetection, np.array(group_size)+1, 'r')
plt.plot(group_size,median_radius,'C1-*',markersize=10, label='median heard')
plt.plot(np.array(group_size)+1,median_notradius,'r-*',markersize=10, label='median unheard')

plt.ylabel('Radius of echo detection \n metres', fontsize=15)
plt.xticks(group_size, group_size, fontsize=12);plt.xlabel('Group size', fontsize=15);
plt.hlines(0.5,4,101, linestyle=':');plt.grid();
plt.text(25,0.40,'Minimum inter-bat distance');plt.xscale('log'); plt.legend()

In [ ]:
plt.figure(figsize=(8,8))
numechoes = [echoesheard[nbats] for nbats in group_size]
median_numechoes = map(np.median, numechoes)
plt.violinplot(numechoes,group_size);
plt.plot(group_size, median_numechoes,'*-',label='median number of echoes')
plt.ylabel(' Number of neighbours detected per call', fontsize=15);plt.grid()
plt.xlabel('Group size',fontsize=15);plt.xscale('log');
plt.xticks(group_size, group_size, fontsize=12);plt.yticks(np.arange(0,22,2))
plt.legend()

In [ ]:
plt.figure(figsize=(8,8))
for nbats in group_size:
    numechoes, freq_heard = np.unique(echoesheard[nbats], return_counts=True)
    p_numechoes = freq_heard/float(np.sum(freq_heard))
    plt.plot(numechoes, p_numechoes, '*-', label='Group size = '+str(nbats));plt.grid()
    plt.text(numechoes[-1]-1,p_numechoes[-1]-0.05, str(nbats)+' bats')
plt.ylabel('Probability of detecting $N_{echoes}$ ',fontsize=15)
plt.xticks(np.arange(0,22,2),fontsize=12);plt.grid();plt.ylim(-0.1,1)
plt.xlabel('Number of echoes detected',fontsize=15);plt.grid()
#plt.legend()